In [17]:

import matplotlib.pyplot as plt
import pickle
import csv
import zipfile

In [18]:
with zipfile.ZipFile('dataset/train.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

In [19]:
with open("./dataset/train.p", mode='rb') as d:
    train = pickle.load(d)
with open("./dataset/valid.p", mode='rb') as d:
    valid = pickle.load(d)
with open("./dataset/test.p", mode='rb') as d:
    test = pickle.load(d)

    
x_train, y_train = train['features'], train['labels']
x_valid, y_valid = valid['features'], valid['labels']
x_testr, y_test = test['features'], test['labels']

n_train = x_train.shape[0]
n_validation = x_valid.shape[0]
n_test = x_testr.shape[0]
image_shape = x_train.shape[1:]
n_classes = len(set(y_train))

In [20]:
import plotly.express as px
import numpy as np

# create the bins
hist, bins = np.histogram(y_train, bins=range(0, 45))
bins = (bins[:-1] + bins[1:])/2

fig = px.bar(x=bins+.5, y=hist, labels={'x':'Classes', 'y':'Sample Count'})
fig.show()

In [21]:
import plotly.graph_objects as go
x=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
accuracy = ['Training Accuracy','Validation Accuracy']
train=[0,14.78,60.07,77.92,83.70,87.69,90.06,91.97,92.97,94.08,95.03,95.67,96.33,96.64,97.01,97.43]
valid= [0,48.91,73.36,76.92,81.68,83.70,86.60,86.21,85.83,86.73,87.57,88.55,88.64,88.41,88.28,88.53]

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=x, y=train, name='Training Accuracy',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x=x, y=valid, name='Validation Accuracy',
                         line=dict(color='royalblue', width=4)))
fig.update_layout(title='Training VS Validation Accuracy',
                   xaxis_title='Epoch',
                   yaxis_title='Percentage(%)')

fig.show()

In [22]:
def gray(img):
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)


def equalize(img):
    return cv2.equalizeHist(img)


def local_histo_equalize(img):
    kernel = morp.disk(30)
    return rank.equalize(img, selem=kernel)

def preprocess(img):
    img = gray(img)
    img =local_histo_equalize(img)
    img = equalize(img)
    #now normalizing the images
    img = img/255
    return img

In [23]:
import cv2
import skimage.morphology as morp
from skimage.filters import rank

x_train= np.array(list(map(preprocess,x_train)))
x_test = np.array(list(map(preprocess,x_testr)))
x_val = np.array(list(map(preprocess,x_valid)))
x_train = x_train.reshape(34799, 32, 32, 1)
x_test = x_test.reshape(12630, 32, 32, 1)
x_val = x_val.reshape(4410, 32, 32, 1)

In [26]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten
cnn_model = Sequential()

cnn_model.add(Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(32,32,1)))
cnn_model.add(AveragePooling2D())
cnn_model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
cnn_model.add(AveragePooling2D())
cnn_model.add(Flatten())
cnn_model.add(Dense(units=400, activation='relu'))
cnn_model.add(Dense(units=84, activation='relu'))
cnn_model.add(Dense(units=43, activation = 'softmax'))
cnn_model.compile(loss ='sparse_categorical_crossentropy', optimizer='adam',metrics =['accuracy'])

In [ ]:
history = cnn_model.fit(x_train,y_train,batch_size=400,epochs=15,verbose=0,validation_data = (x_val,y_valid))

In [ ]:
#get the predictions for the test data
predicted_classes = np.argmax(cnn_model.predict(x_test), axis=-1)
#get the indices to be plotted
y_true = y_test

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, predicted_classes)
plt.figure(figsize = (20,20))
sns.heatmap(cm, annot=True)
